In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Pending

In [ ]:
# from scipy.io import loadmat

# link = r"D:\external_data\Experiment4"
# filename = r"\Dataset_PWR_WiFi.mat"
# directory = link + filename

# mat = loadmat(directory)

In [ ]:
# # import experimental dataset 2
# folderpath2 = "D:/external_data/Experiment3/csv_files/exp_2"  # CHANGE THIS IF THE PATH CHANGED
# df_exp2 = import_clean_data('exp2',folderpath2)

# lab

### visualisation

In [ ]:
from data.import_data import import_experimental_data

# 1. Import data
folderpath1 = "E:/external_data/Experiment3/csv_files/exp_1"
df = import_clean_data('exp1',folderpath1)


In [ ]:
from models.train import seperate_dataframes
from data.process_data import DatasetObject, slide_augmentation
from data import process_data


X, y = seperate_dataframes(df_exp1)

datasetobj = DatasetObject()
datasetobj.import_data(X, y)
del X, y

In [ ]:

window_size = 900
slide_size = 200
datasetobj.data_transform(lambda x,y,z : process_data.slide_augmentation(x, y, z,
                                                            window_size=window_size,
                                                            slide_size=slide_size,
                                                            skip_labels=['noactivity']),axis=0)
# datasetobj.data_transform(lambda arr: arr.reshape(-1,window_size,3,30).transpose(0,2,1,3),axis=1, col=0)
datasetobj.data_transform(lambda arr: arr.reshape(-1,window_size,3,30).transpose(0,2,3,1),axis=1, col=0)
datasetobj.shape()

In [ ]:
X,y = datasetobj.data[0,0],datasetobj.data[0,1]


num = 102
channel = 0
X,y = X[num][channel],y[num]

# channel = 0
# X = X[channel]
# X = X.reshape(900,30,3)
# X = X.transpose(2,0,1)[0]

for i in [1,3,5,10,30,60]:
    X_ = X[::3,::i]

    print(X_.shape)
    plt.figure(figsize=(15,20))
    plt.imshow(X_,cmap='jet')
    plt.title(y)

In [ ]:
a = X[:,60:90]
print(a.shape)
a.transpose()

In [ ]:
from data.process_data import breakpoints

def breakpoints(ls):
    dic = {}
    for i in range(len(ls)-1):
        if ls[i+1] != ls[i]:
            string = f"{ls[i]}-{ls[i+1]}"
            dic[string] = i
    return dic

points = breakpoints(y_ls[0].reshape(-1).tolist())
print(points)
print('cutpoint: ',y_ls[0].shape[0]//8)

In [ ]:
def normalise():

In [ ]:
user1 = X_ls[0].transpose()
user1_c = user1.reshape(3,30,-1)
user1_bendfwd_ = user1[:,:6999]
user1_kneel_   = user1[:,6999:6999+6999]

In [ ]:
plt.figure(figsize = (15,50))
p = 3000
plt.imshow(user1_c[1][:,p:p+2000],cmap='jet')

In [ ]:
plt.figure(figsize = (15,50))
p = 3000
sample = user1[:,p:p+2000]
# MinMaxScaler().fit_transform(sample)
plt.imshow(sample,cmap='jet')

In [ ]:
y_ls[0][p:p+6999,:]

In [ ]:
user1_X = X_ls[0]
user1_y = y_ls[0]

# Section
# p = 6999*1

for p in range(0,6999*8,6999):

    sample = user1[p:p+6999,:]



    # Scale along first axis 
    # sample = MinMaxScaler().fit_transform(sample)

    # Transpose 
    sample = sample.transpose()

    # ImShow
    plt.figure(figsize = (25,150))
    plt.imshow(sample,cmap='jet')

In [ ]:
6999*2

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_ls[0].shape

In [ ]:
# Visualise spectrogram
img = img.transpose()
img = img[:,::10]
plt.figure(figsize = (15,50))
plt.imshow(img,cmap='jet')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from data.process_data import stacking
import matplotlib.pyplot as plt



def visual_spectrogram(img,title='spectrogram'):
    
    if len(img.shape) == 3:
        
        img.squeeze()
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(img)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
    plt.figure(figsize = (15,50))
    plt.imshow(img,cmap='jet')
    plt.title(title)
    plt.show()
    
    return 

In [ ]:
import torchvision.models as models
        
#visualize weights for alexnet - first conv layer
alexnet = models.alexnet(pretrained=False)

# Store all conv layer in alexnet
k = []
for m in alexnet.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data.shape)
        k.append(m)
    else:  
        continue


In [ ]:
def visual_single_filter(weight):
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(weight)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
#     plt.figure(figsize = (15,10))
    plt.imshow(weight,cmap='jet')
#     plt.show()
    
    return 

def visualise_filters(layer,figsize=(20,15)):
    
    weights = layer.weight.data

    if weights.shape[0] > weights.shape[1]:
        
        weights = weights.transpose(1,0)
        
    rows = weights.shape[0]
    
    cols = weights.shape[1]
    
    axes = []
    
    fig= plt.figure()

    for n in range(rows):

        for c in range(cols):
            
            axes.append(fig.add_subplot(rows, cols, n*cols+c+1))

            single_filter = weights[n,c]
        
            visual_single_filter(single_filter)
            
#     fig.tight_layout()    
    
    plt.show()     
    
    return 

visualise_filters(example)

# model factory

In [1]:
import time
from time import gmtime, strftime
# a = time.time()
# model  = train(model, train_loader, criterion, optimizer, 48)
# print(time.time() - a)

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

In [3]:
from models.baseline import Lambda

##### extra setting

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

torch.cuda.device_count()

torch.cuda.set_device(device)

cuda:0


In [5]:
np.random.seed(1024)
torch.manual_seed(1024)
# torch.set_deterministic(True)

### 0. Default

In [6]:
# device = torch.device("cpu")
# torch.cuda.set_device(device)

In [7]:
# model 
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        pass
    
    def forward(self,X):
        return X

### 1. Section

### a. Residual

In [8]:
class Residual_Block(nn.Module):
    """
    Single channel for radio image (30,30,1)
    """
    def __init__(self,in_feature,out_feature,kernel_size,stride, padding='zeros', bias=False):
        super(Residual_Block, self).__init__()
        ### 1st ###
        self.conv1 = nn.Conv2d(in_feature,out_feature,kernel_size=kernel_size,stride=stride, bias=bias)
        self.norm1 = nn.BatchNorm2d(out_feature)
        self.actv1 = nn.ReLU(inplace=True)
        self.pool1 = nn.AvgPool2d(kernel_size=kernel_size,stride=stride)
        self.covr1 = nn.Conv2d(in_feature,out_feature,kernel_size=1,stride=1, bias=False)
        ### 2nd ###
        self.conv2 = nn.Conv2d(out_feature,out_feature,kernel_size=kernel_size,stride=stride, bias=bias)
        self.norm2 = nn.BatchNorm2d(out_feature)
        self.actv2 = nn.ReLU(inplace=True)
        self.pool2 = nn.AvgPool2d(kernel_size=kernel_size,stride=stride)
        
        self.covr1.weight.requires_grad = False
        
    def forward(self,X):
        R = X
        ### 1st ###
        X = self.actv1(self.norm1(self.conv1(X)))
        R = self.covr1(self.pool1(R))
        ### 2nd ###
        X = self.norm2(self.conv2(X))
        R = self.pool2(R)
        # print(X.shape,R.shape)
        X += R
        X = self.actv2(X)
        return X

    
class Residual(nn.Module):
    
    def __init__(self):
        super(Residual, self).__init__()
        self.norm = nn.BatchNorm2d(1)
        self.block1 = Residual_Block(in_feature=  1, out_feature=  64, kernel_size = (5,5), stride = (1,2))
        self.block2 = Residual_Block(in_feature= 64, out_feature= 128, kernel_size = (4,4), stride = (2,2))
        self.block3 = Residual_Block(in_feature=128, out_feature= 256, kernel_size = (3,3), stride = (2,2))
        self.pool = nn.AdaptiveAvgPool2d((1,3))
    
    def forward(self,x):
        x = self.norm(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(self.pool(x), 1)
        return x
  

In [6]:
class Encoder_View(nn.Module):
    """

    This model first divides a spectrogram into number of pieces based on window and channel,
    each piece is processed with the single encoder and output a matrix with shape (batch_size, seq_size, num_features),
    input shape: (batch_size,num_frame,num_channel)
    """
    def __init__(self, encoder=None):
        super(Encoder_View, self).__init__()
        self.encoder = encoder

    def forward(self,x):
        ### Method 1 ###
#         size = x.shape
#         x = x.view(-1,1,30,900)
#         x = self.encoder(x)
#         x = x.view(size[0],-1)
        ### Method 2 ###
        a,b,c = torch.split(x,1,dim=1)
        a = self.encoder(a)
        b = self.encoder(b)
        c = self.encoder(c)
        x = (a+b+c)/3
        return x

# a = time.time()
# sample = torch.rand(size=(64,3,30,900))
# full_encoder = Encoder_View(encoder=Encoder())
# sample = full_encoder(sample)
# sample.shape
# print(time.time()-a)

### b. Shallow

In [7]:
class Encoder(nn.Module):
    """
    4 layers
    """
    def __init__(self):
        super(Encoder, self).__init__()
        ### 1st ###
        self.conv1 = nn.Conv2d(1,32,kernel_size=(3,3),stride=(1,1),padding=0)
        self.actv1 = nn.ReLU()
        self.norm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.AdaptiveMaxPool2d((20,300))
        ### 2nd ###
        self.conv2 = nn.Conv2d(32,64,kernel_size=(3,3),stride=(1,1),padding=0)
        self.actv2 = nn.ReLU()
        self.norm2 = nn.BatchNorm2d(64)
        self.pool2 = nn.AdaptiveMaxPool2d((10,100))
        ### 3rd ###
        self.conv3 = nn.Conv2d(64,128,kernel_size=(3,3),stride=(1,1),padding=0)
        self.actv3 = nn.ReLU()
        self.norm3 = nn.BatchNorm2d(128)
        self.pool3 = nn.AdaptiveMaxPool2d((5,30))
        ### 4rd ###
        self.conv4 = nn.Conv2d(128,256,kernel_size=(3,3),stride=(1,1),padding=0)
        self.actv4 = nn.Tanh()
        self.norm4 = Lambda(lambda x:x)
        self.pool4 = nn.AdaptiveAvgPool2d((2,4))
        
    def forward(self,X):
        X = self.pool1(self.actv1(self.norm1(self.conv1(X))))
        X = self.pool2(self.actv2(self.norm2(self.conv2(X))))
        X = self.pool3(self.actv3(self.norm3(self.conv3(X))))
        X = self.pool4(self.actv4(self.norm4(self.conv4(X))))
        X = torch.flatten(X, 1)
        return X
    
summary(Encoder().cuda(),input_size=(1,30,900))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 28, 898]             320
       BatchNorm2d-2          [-1, 32, 28, 898]              64
              ReLU-3          [-1, 32, 28, 898]               0
 AdaptiveMaxPool2d-4          [-1, 32, 20, 300]               0
            Conv2d-5          [-1, 64, 18, 298]          18,496
       BatchNorm2d-6          [-1, 64, 18, 298]             128
              ReLU-7          [-1, 64, 18, 298]               0
 AdaptiveMaxPool2d-8          [-1, 64, 10, 100]               0
            Conv2d-9           [-1, 128, 8, 98]          73,856
      BatchNorm2d-10           [-1, 128, 8, 98]             256
             ReLU-11           [-1, 128, 8, 98]               0
AdaptiveMaxPool2d-12           [-1, 128, 5, 30]               0
           Conv2d-13           [-1, 256, 3, 28]         295,168
           Lambda-14           [-1, 256

### c.SimCLR 

In [58]:
class Projection_Head(nn.Module):
    
    def __init__(self,feature_in,feature_out):
        super(Projection_Head, self).__init__()
        # self.layer1 = nn.Linear(feature_in,feature_in)
        self.layer2 = nn.Linear(feature_in,feature_out)
    
    def forward(self,X):
        # X = self.layer1(X)
        X = self.layer2(X)
        return X

class SimCLR_TXR_model(nn.Module):
    def __init__(self, encoder, decoder):
        super(SimCLR_TXR_model, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self,x):
        a,b,c = torch.split(x,1,dim=1)
        a = self.encoder(a)
        b = self.encoder(b)
        c = self.encoder(c)
        a = self.decoder(a)
        b = self.decoder(b)
        c = self.decoder(c)
        return a,b,c

# encoder = Encoder()
# decoder = Projection_Head(768,128)
# model = SimCLR_TXR_model(encoder=encoder,decoder=decoder)
# summary(model=model.cuda(),input_size=(3,30,900),batch_size=64)

In [59]:
class Contrastive_multiview_loss(nn.Module):
    
    def __init__(self,loss_func):
        super(Contrastive_multiview_loss, self).__init__()
        self.loss_func = loss_func 
        
    def forward(self,*vecs):
        loss = sum([self.loss_func(vecs[i],vecs[j]) for i in range(0,len(vecs)-1) for j in range(i+1,len(vecs))])
#         loss = torch.tensor(0)
#         for i in range(0,len(vecs)-1):
#             for j in range(i+1,len(vecs)):
#                 cur = self.loss_func(vecs[i],vecs[j])
#                 loss = loss.add(cur)
        return loss

    
# encoder = Encoder()
# decoder = Projection_Head(768,128)
# model = SimCLR_TXR_model(encoder=encoder,decoder=decoder)
# loss_func = NT_Xent(64,1)
# multiview_loss = Contrastive_multiview_loss(loss_func)


# sample = torch.rand((64,3,30,900))
# a,b,c = model(sample)
# loss = multiview_loss(a,b,c)
# loss

### 2. calling function

In [60]:
### Pretraining
# def create_model():
#     encoder = Encoder()
#     decoder = Projection_Head(2048,128)
#     model = SimCLR_TXR_model(encoder=encoder,decoder=decoder)
#     return model

# def setting(model):
#     """
#     NTXent
#     """
# #     criterion = NT_Xent(batch_size=64,temperature=0.1)
#     criterion = Contrastive_multiview_loss(loss_func=NT_Xent(128,1))
#     optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
#     return criterion, optimizer


# model = create_model()
# model = create_model().to(device)
# summary(create_model().cuda(),(3,30,900))

In [9]:
from models.baseline import Classifier,CNN_module

In [10]:
def create_model():
    """
    V3
    """
    encoder = Encoder_View(encoder=Encoder())
    decoder = Classifier(2048)
    model = CNN_module(encoder=encoder,decoder=decoder)
    return model

summary(create_model(),input_size=(3,30,900),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 32, 28, 898]             320
       BatchNorm2d-2          [-1, 32, 28, 898]              64
              ReLU-3          [-1, 32, 28, 898]               0
 AdaptiveMaxPool2d-4          [-1, 32, 20, 300]               0
            Conv2d-5          [-1, 64, 18, 298]          18,496
       BatchNorm2d-6          [-1, 64, 18, 298]             128
              ReLU-7          [-1, 64, 18, 298]               0
 AdaptiveMaxPool2d-8          [-1, 64, 10, 100]               0
            Conv2d-9           [-1, 128, 8, 98]          73,856
      BatchNorm2d-10           [-1, 128, 8, 98]             256
             ReLU-11           [-1, 128, 8, 98]               0
AdaptiveMaxPool2d-12           [-1, 128, 5, 30]               0
           Conv2d-13           [-1, 256, 3, 28]         295,168
           Lambda-14           [-1, 256

In [36]:
del model
torch.cuda.empty_cache()

In [14]:
assert False

AssertionError: 

# Test

In [11]:
from data import process_data
from data.import_data import import_clean_data
from data.datasetobj import DatasetObject
# from models.train import * 

### 0. shortcut

In [12]:
# from models.train import prepare_data

# train_loader, test_loader = prepare_data()

### 1. Data

In [12]:
# 1. Import data
fp = "E:/external_data/Experiment3/csv_files/exp_1"
df = import_clean_data('exp_1',fp)

Found 80 files.
input_user10_bendfwd.csv annotation_user10_bendfwd.csv user10
input_user10_kneel.csv annotation_user10_kneel.csv user10
input_user10_lie.csv annotation_user10_lie.csv user10
input_user10_sit.csv annotation_user10_sit.csv user10
input_user10_sitrotate.csv annotation_user10_sitrotate.csv user10
input_user10_stand.csv annotation_user10_stand.csv user10
input_user10_standrotate.csv annotation_user10_standrotate.csv user10
input_user10_walking.csv annotation_user10_walking.csv user10
input_user1_bendfwd.csv annotation_user1_bendfwd.csv user1
input_user1_kneel.csv annotation_user1_kneel.csv user1
input_user1_lie.csv annotation_user1_lie.csv user1
input_user1_sit.csv annotation_user1_sit.csv user1
input_user1_sitrotate.csv annotation_user1_sitrotate.csv user1
input_user1_stand.csv annotation_user1_stand.csv user1
input_user1_standrotate.csv annotation_user1_standrotate.csv user1
input_user1_walking.csv annotation_user1_walking.csv user1
input_user2_bendfwd.csv annotation_user2

In [13]:
# 2. Process data
from sklearn.preprocessing import MinMaxScaler

def seperate_dataframes(df):
    features_ls,labels_ls = [],[]
    for user in df['user'].unique():
        dataframe = df[df['user']==user]
        features = dataframe[[f'amp_{i}' for i in range(1,91)]].to_numpy()
        features = MinMaxScaler().fit_transform(features)
        features_ls.append(features)
        label = dataframe[['label']].to_numpy()
        labels_ls.append(label)
    return features_ls,labels_ls


X_ls, y_ls = seperate_dataframes(df)
del df

In [14]:
def create_datasetobj(X,y):
    datasetobj = DatasetObject()
    datasetobj.import_data(X, y)
    return datasetobj

datasetobj = create_datasetobj(X_ls,y_ls)

In [15]:
from sklearn.preprocessing import LabelEncoder

def transform_datasetobj(datasetobj):
    window_size = 900
    slide_size = 200
    datasetobj.data_transform(lambda x,y,z : process_data.slide_augmentation(x, y, z,
                                                                window_size=window_size,
                                                                slide_size=slide_size,
                                                                skip_labels=['noactivity']),axis=0)
    datasetobj.data_transform(lambda arr: arr.reshape(-1,window_size,3,30).transpose(0,2,3,1),axis=1, col=0)
    datasetobj.data_transform(lambda x,y,z : process_data.resampling(x, y, z, oversampling = True),axis=0)
    label_encoder = LabelEncoder()
    label_encoder.fit(datasetobj()[1])
    datasetobj.data_transform(lambda arr: label_encoder.transform(arr).reshape(arr.shape),axis=1, col=1)
    return datasetobj, label_encoder

datasetobj, label_encoder = transform_datasetobj(datasetobj)
datasetobj.shape()

index 0 arrays sizes ------ X:  (192, 3, 30, 900)  Y:  (192,)  Z:  (192, 1)
index 1 arrays sizes ------ X:  (208, 3, 30, 900)  Y:  (208,)  Z:  (208, 1)
index 2 arrays sizes ------ X:  (216, 3, 30, 900)  Y:  (216,)  Z:  (216, 1)
index 3 arrays sizes ------ X:  (184, 3, 30, 900)  Y:  (184,)  Z:  (184, 1)
index 4 arrays sizes ------ X:  (192, 3, 30, 900)  Y:  (192,)  Z:  (192, 1)
index 5 arrays sizes ------ X:  (176, 3, 30, 900)  Y:  (176,)  Z:  (176, 1)
index 6 arrays sizes ------ X:  (192, 3, 30, 900)  Y:  (192,)  Z:  (192, 1)
index 7 arrays sizes ------ X:  (176, 3, 30, 900)  Y:  (176,)  Z:  (176, 1)
index 8 arrays sizes ------ X:  (184, 3, 30, 900)  Y:  (184,)  Z:  (184, 1)
index 9 arrays sizes ------ X:  (200, 3, 30, 900)  Y:  (200,)  Z:  (200, 1)
size of DatasetObject ------ :  (10, 3)


In [16]:
del X_ls, y_ls

In [17]:
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset

def create_dataloaders(X_train, y_train, X_test, y_test, batch_sizes={'train':64, 'test':200}):
    traindataset = TensorDataset(Tensor(X_train),Tensor(y_train).long())
    testdataset = TensorDataset(Tensor(X_test), Tensor(y_test).long())
    train_loader = DataLoader(traindataset, batch_size=batch_sizes['train'], shuffle=True, num_workers=4, drop_last=True)
    test_loader = DataLoader(testdataset, batch_size=batch_sizes['test'], shuffle=True, num_workers=4)
    return train_loader, test_loader

(X_train, y_train,_),(X_test, y_test,_) = datasetobj([9],return_train_sets=True)
train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test)

train set: [0, 1, 2, 3, 4, 5, 6, 7, 8] 	test set: [9]


In [29]:
del X_train, y_train, X_test, y_test

### 2. Model

In [20]:
# from models.baseline import Lambda, Classifier,CNN_module

In [21]:
# del model ,criterion , optimizer
# torch.cuda.empty_cache()

In [18]:
model = create_model()

In [19]:
def setting(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.0005)
    return criterion, optimizer

criterion, optimizer = setting(model)

### 3. Training

In [20]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [21]:
def train(model, train_loader, criterion, optimizer, end, start = 1, test_loader = None, parallel = None, **kwargs):

    # Check device setting
    if parallel == True:
        print('GPU')
        model = model.cuda()
    else:
        print('CPU')

    print('Start Training')
    record = {'train':[],'validation':[]}
    i = start
    #Loop
    while i <= end:
        print(f"Epoch {i}: ", end='')
        for b, (X_train, y_train) in enumerate(train_loader):
            if parallel == True:
                X_train, y_train = X_train.cuda(), y_train.cuda() #.to(device)
            print(f">", end='')
            optimizer.zero_grad()
            y_pred = model(X_train)
            loss   = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()
            del X_train, y_train, y_pred
        # One epoch completed
        loss = loss.tolist()
        record['train'].append(loss)
        print(f' loss: {loss} ',end='')
        if (test_loader != None) and i%10 ==0 :
            acc = short_evaluation(model,test_loader,parallel)
            record['validation'].append(acc)
            print(f' accuracy: {acc}')
        else:
            print('')
        i += 1

    model = model.cpu()
    return model, record

def short_evaluation(model,test_loader,parallel):
    # copy the model to cpu
    if parallel == True:
        model = model.cpu()
    with torch.no_grad():
        for X_test, y_test in test_loader:
            y_val = model(X_test)
            predicted = torch.max(y_val, 1)[1]
            acc = accuracy_score(y_test.view(-1), predicted.view(-1))
    # send model back to gpu
    if parallel == True:
        model = model.cuda()
    return acc

model = train(model, train_loader, criterion, optimizer, end = 300, start = 1, 
              test_loader = test_loader, parallel = True)

GPU
Start Training
Epoch 1: >

RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 4.00 GiB total capacity; 2.81 GiB already allocated; 28.84 MiB free; 2.85 GiB reserved in total by PyTorch)

In [ ]:
assert False

### 4. Testing

In [ ]:
def evaluation(model,test_loader):
    model = model.cpu()
    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test, y_test
            y_val = model(X_test)
            predicted = torch.max(y_val, 1)[1]
    cls = classification_report(y_test.view(-1), predicted.view(-1), output_dict=True)
    cls = pd.DataFrame(cls)
    print(cls)
    cmtx = confusion_matrix(y_test.view(-1), predicted.view(-1))
    return cmtx,cls

def cmtx_table(cmtx,label_encoder=None):
    if label_encoder != None:
        cmtx = pd.DataFrame(cmtx,
                            index=[f"actual: {i}"for i in label_encoder.categories_[0].tolist()], 
                            columns=[f"predict : {i}"for i in label_encoder.categories_[0].tolist()])
    else: 
        cmtx = pd.DataFrame(cmtx)
    return cmtx

cmtx,cls = evaluation(model,test_loader)
cmtx = cmtx_table(cmtx,label_encoder=None)
cmtx

In [ ]:
train_loss = pd.DataFrame(record['train'],columns=['train_loss'])

### 5. Saving

In [ ]:
from models.train import save_checkpoint, make_directory

In [ ]:
path = make_directory('V2cSimCLR',epoch=500,filepath='./record/')
train_loss.to_csv(path+'_loss.csv')
# cls.to_csv(path+'_report.csv')
# cmtx.to_csv(path+'_cmtx.csv')

In [ ]:
path = make_directory('V2cSimCLR_pretrain',epoch=500)
save_checkpoint(model, optimizer, 500, path)